# Game Data Crawling

## Jul

In [1]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options) #크롬드라이버 따로 설치 필요

In [3]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('07')
time.sleep(1)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [4]:
jul = pd.read_html(str(soup.select('div.tbl-type06')))[0].iloc[90:, :3].reset_index(drop=True)

In [5]:
a = pd.DataFrame(columns=['날짜', '시간', '경기'])
for i in range(jul.shape[0]):
    a = pd.concat([a, pd.DataFrame([jul.iloc[i, :]]), pd.DataFrame([jul.iloc[i, :]])])
jul = a.reset_index(drop=True)
jul = jul.assign(AB=0, PA=0, RBI=0, RUN=0, HIT=0, H2=0, H3=0, HR=0, SB=0, CS=0, SH=0, SF=0, BB=0, IB=0,
           HP=0, KK=0, GD=0, ERR=0, CG_CK=0, WLS=0, HOLD=0, WP=0, BK=0, INN2=0, BF=0, ER=0, R=0)

In [6]:
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)
driver.get('https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx')
driver.find_element_by_css_selector('#contents > div.date-select > ul > li.today > img').click()
driver.find_element_by_css_selector('#ui-datepicker-div > div > div > select.ui-datepicker-month').send_keys('7월(JUL)')
driver.find_element_by_css_selector('#ui-datepicker-div > table > tbody > tr:nth-child(4) > td:nth-child(2) > a').click()

n = 0

for j in range(11):
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    x = len(list(soup.find_all('div', {'class':'game-cont'})))
    
    for i in range(1, x+1):
        try:
            driver.find_element_by_css_selector('#contents > div.today-game > div > div.bx-viewport > ul > li:nth-child(%d) > div' % i).click()
            driver.find_element_by_css_selector('#tabDepth2 > li:nth-child(2) > a').click()
            
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            table = pd.read_html(str(soup.select('div.summary.tbl-type02.mt30.mb30')))[0]
            table.index = table[0]
            table = table.drop([0], axis=1)
            table = table.T.reset_index(drop=True)
            
            T_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[0],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[1],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[2]], axis=1).iloc[:, 2:]
            
            B_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[3],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[4],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[5]], axis=1).iloc[:, 2:]
            
            T_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[0]
            
            B_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[1]
         
            jul.iloc[n, 3] = int(T_hit.loc[T_hit['선수명'].isnull(), '타수'])
            jul.iloc[n, 5] = int(T_hit.loc[T_hit['선수명'].isnull(), '타점'])
            jul.iloc[n, 6] = int(T_hit.loc[T_hit['선수명'].isnull(), '득점'])
            jul.iloc[n, 7] = int(T_hit.loc[T_hit['선수명'].isnull(), '안타'])
            jul.iloc[n+1, 3] = int(B_hit.loc[B_hit['선수명'].isnull(), '타수'])
            jul.iloc[n+1, 5] = int(B_hit.loc[B_hit['선수명'].isnull(), '타점'])
            jul.iloc[n+1, 6] = int(B_hit.loc[B_hit['선수명'].isnull(), '득점'])
            jul.iloc[n+1, 7] = int(B_hit.loc[B_hit['선수명'].isnull(), '안타'])
            
            for i in range(T_hit.shape[0]):
                for j in range(1, 10):
                    if T_hit.iloc[i, j] == '4구':
                        jul.iloc[n, 15] += 1
                    elif T_hit.iloc[i, j] == '고4':
                        jul.iloc[n+1, 16] += 1
                    elif T_hit.iloc[i, j] == '사구':
                        jul.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('H2')] += 1
                    elif re.search('[가-힣]3', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('H3')] += 1
                    elif re.search('[가-힣]홈', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(T_hit.iloc[i,j])) != None:
                        jul.iloc[n, jul.columns.get_loc('GD')] += 1
            
            for i in range(B_hit.shape[0]):
                for j in range(1, 10):
                    if B_hit.iloc[i, j] == '4구':
                        jul.iloc[n+1, 15] += 1
                    elif B_hit.iloc[i, j] == '고4':
                        jul.iloc[n+1, 16] += 1
                    elif B_hit.iloc[i, j] == '사구':
                        jul.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('H2')] +=  1
                    elif re.search('[가-힣]3', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('H3')] +=  1
                    elif re.search('[가-힣]홈', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(B_hit.iloc[i,j])) != None:
                        jul.iloc[n+1, jul.columns.get_loc('GD')] += 1
            
            if '도루' in table.columns:
                for i in range(len(table['도루'][0].split(' '))):
                    if table['도루'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        jul.iloc[n, 11] += 1
                    elif table['도루'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        jul.iloc[n+1, 11] += 1
            else: pass
            
            if '도루자' in table.columns:
                for i in range(len(table['도루자'][0].split(' '))):
                    if table['도루자'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        jul.iloc[n, 12] = jul.iloc[n, 12] + 1
                    elif table['도루자'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        jul.iloc[n+1, 12] = jul.iloc[n+1, 12] + 1
            else: pass
            
            if '실책' in table.columns:
                for i in range(len(table['실책'][0].split(' '))):
                    if table['실책'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        jul.iloc[n, 20] += 1
                    elif table['실책'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        jul.iloc[n+1, 20] += 1
            
            # CG_CK
            if T_pit.shape[0] == 2:
                jul.iloc[n, 21] = 1
            else: pass
            if B_pit.shape[0] == 2:
                jul.iloc[n+1, 21] = 1
            else: pass
            
            if '승' in list(T_pit.iloc[:,2]):
                jul.iloc[n, 22] = 'W'
                jul.iloc[n+1, 22] = 'L'
            elif '패' in list(T_pit.iloc[:,2]):
                jul.iloc[n, 22] = 'L'
                jul.iloc[n+1, 22] = 'W'
            else:
                jul.iloc[n, 22] = 'D'
                jul.iloc[n+1, 22] = 'D'
            
            if '폭투' in table.columns:
                for i in range(len(table['폭투'][0].split(' '))):
                    if table['폭투'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        jul.iloc[n, 24] += 1
                    elif table['폭투'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        jul.iloc[n+1, 24] += 1
            
            if '보크' in table.columns:
                for i in range(len(table['보크'][0].split(' '))):
                    if table['보크'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        jul.iloc[n, 25] += 1
                    elif table['보크'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        jul.iloc[n+1, 25] += 1  
            
            if '홀드' in list(T_pit.iloc[:,2]):
                    jul.iloc[n, 23] = list(T_pit.iloc[:,2]).count('홀드')
            if '홀드' in list(B_pit.iloc[:,2]):
                jul.iloc[n+1, 23] = list(B_pit.iloc[:,2]).count('홀드')
            
            # KK
            jul.iloc[n, 18] = T_pit.iloc[-1, 13]
            jul.iloc[n+1, 18] = B_pit.iloc[-1, 13]
            
            # INN2
            jul.iloc[n, 26] = T_pit.iloc[-1, 6]
            jul.iloc[n+1, 26] = B_pit.iloc[-1, 6]
            
            # BF
            jul.iloc[n, 27] = T_pit.iloc[-1, 8]
            jul.iloc[n+1, 27] = B_pit.iloc[-1, 8]
            
            # ER
            jul.iloc[n, 28] = T_pit.iloc[-1, 15]
            jul.iloc[n+1, 28] = B_pit.iloc[-1, 15]
            
            # R
            jul.iloc[n, 29] = T_pit.iloc[-1, 14]
            jul.iloc[n+1, 29] = B_pit.iloc[-1, 14]
            
            # PA
            jul.iloc[n, 4] = B_pit.iloc[-1, 7]
            jul.iloc[n+1, 4] = T_pit.iloc[-1, 7]
            
            # AB
            jul.iloc[n, 3] = B_pit.iloc[-1, 9]
            jul.iloc[n+1, 3] = T_pit.iloc[-1, 9]
            
        except: pass
        
        n += 2

    driver.find_element_by_css_selector('#lnkNext > img').click()

In [7]:
jul_copy = jul.copy()

In [8]:
for i in range(jul_copy.shape[0]-10):
    for j in range(1, 11):
        if sum(jul_copy.iloc[i, 3:] == jul_copy.iloc[i+j, 3:]) == 27:
            jul_copy.iloc[i+j, 3:] = 0

In [9]:
jul_copy_a = jul_copy.iloc[[2*n for n in range(int(jul_copy.shape[0]/2))],].reset_index(drop=True)
jul_copy_b = jul_copy.iloc[[2*n+1 for n in range(int(jul_copy.shape[0]/2))],].reset_index(drop=True)

In [10]:
jul_copy_a = jul_copy_a.assign(T=None, B=None)
jul_copy_b = jul_copy_b.assign(T=None, B=None)

In [11]:
for i in range(jul_copy_a.shape[0]):
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('T')] = jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('경기')].split('vs')[0][:2]
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('B')] = jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('경기')].split('vs')[1][-2:]
for i in range(jul_copy_b.shape[0]):
    jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('T')] = jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('경기')].split('vs')[0][:2]
    jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('B')] = jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('경기')].split('vs')[1][-2:]

In [12]:
for i in range(jul_copy_a.shape[0]):
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('날짜')] = '2020.' + jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('날짜')][:5]
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('날짜')] = jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('날짜')].replace('.', '')
for i in range(jul_copy_b.shape[0]):
    jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('날짜')] = '2020.' + jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('날짜')][:5]
    jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('날짜')] = jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('날짜')].replace('.', '')

In [13]:
name = {'NC':'NC', '두산':'OB', '키움':'WO', 'KI':'HT', 'IA':'HT', 'LG':'LG',
       '삼성':'SS', 'KT':'KT', '롯데':'LT', 'SK':'SK', '한화':'HH'}

f = lambda x: name.get(x, x)
jul_copy_a['T'] = jul_copy_a['T'].map(f) ; jul_copy_a['B'] = jul_copy_a['B'].map(f)
jul_copy_b['T'] = jul_copy_b['T'].map(f) ; jul_copy_b['B'] = jul_copy_b['B'].map(f)

In [14]:
jul_copy_a['G_ID'] = jul_copy_a['날짜'] + jul_copy_a['T'] + jul_copy_a['B']
jul_copy_b['G_ID'] = jul_copy_b['날짜'] + jul_copy_b['T'] + jul_copy_b['B']

In [15]:
for i in jul_copy_a[jul_copy_a.duplicated(['G_ID'], keep=False) == True].index:
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')] = jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')] + '1'
for i in jul_copy_a[jul_copy_a.duplicated(['G_ID'], keep=False) == False].index:
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')] = jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')] + '0'
for i in jul_copy_b[jul_copy_b.duplicated(['G_ID'], keep=False) == True].index:
    jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')] = jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')] + '1'
for i in jul_copy_b[jul_copy_b.duplicated(['G_ID'], keep=False) == False].index:
    jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')] = jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')] + '0'

In [16]:
jul_copy_a = jul_copy_a.drop(['시간', '경기'], axis=1)
jul_copy_b = jul_copy_b.drop(['시간', '경기'], axis=1)

In [17]:
jul_copy_a = jul_copy_a.rename(columns={'T':'T_ID', 'B':'VS_T_ID'})
jul_copy_b = jul_copy_b.rename(columns={'T':'VS_T_ID', 'B':'T_ID'})

In [18]:
jul_copy_a = jul_copy_a.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)
jul_copy_b = jul_copy_b.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)

In [19]:
for i in range(jul_copy_a.shape[0]):
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('GDAY_DS')] = jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')][:8]
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('HEADER_NO')] = jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')][12]
    jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('TB_SC')] = 'T'
for i in range(jul_copy_b.shape[0]):
    jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('GDAY_DS')] = jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('G_ID')][:8]
    jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('HEADER_NO')] = jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('G_ID')][12]
    jul_copy_b.iloc[i, jul_copy_a.columns.get_loc('TB_SC')] = 'B'

In [20]:
for i in range(jul_copy_a.shape[0]-1):
    if (jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')][-1] == '1') & (jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')] == jul_copy_a.iloc[i+1, jul_copy_a.columns.get_loc('G_ID')]):
        jul_copy_a.iloc[i+1, jul_copy_a.columns.get_loc('G_ID')] = jul_copy_a.iloc[i+1, jul_copy_a.columns.get_loc('G_ID')].replace('1', '2')
        jul_copy_a.iloc[i+1, jul_copy_a.columns.get_loc('HEADER_NO')] = '2'
    elif (jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')][-1] == '1') & (jul_copy_a.iloc[i, jul_copy_a.columns.get_loc('G_ID')] != jul_copy_a.iloc[i+1, jul_copy_a.columns.get_loc('G_ID')]):
        jul_copy_a.iloc[i+2, jul_copy_a.columns.get_loc('G_ID')] = jul_copy_a.iloc[i+2, jul_copy_a.columns.get_loc('G_ID')].replace('1', '2')
        jul_copy_a.iloc[i+2, jul_copy_a.columns.get_loc('HEADER_NO')] = '2'

for i in range(jul_copy_b.shape[0]-1):
    if (jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')][-1] == '1') & (jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')] == jul_copy_b.iloc[i+1, jul_copy_b.columns.get_loc('G_ID')]):
        jul_copy_b.iloc[i+1, jul_copy_b.columns.get_loc('G_ID')] = jul_copy_b.iloc[i+1, jul_copy_b.columns.get_loc('G_ID')].replace('1', '2')
        jul_copy_b.iloc[i+1, jul_copy_b.columns.get_loc('HEADER_NO')] = '2'
    elif (jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')][-1] == '1') & (jul_copy_b.iloc[i, jul_copy_b.columns.get_loc('G_ID')] != jul_copy_b.iloc[i+1, jul_copy_b.columns.get_loc('G_ID')]):
        jul_copy_b.iloc[i+2, jul_copy_b.columns.get_loc('G_ID')] = jul_copy_b.iloc[i+2, jul_copy_b.columns.get_loc('G_ID')].replace('1', '2')
        jul_copy_b.iloc[i+2, jul_copy_b.columns.get_loc('HEADER_NO')] = '2'

In [21]:
jul_copy = pd.concat([jul_copy_a, jul_copy_b]).sort_values('G_ID').reset_index(drop=True)

In [22]:
jul_copy_hit = jul_copy[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'PA', 'AB',
       'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
       'IB', 'HP', 'KK', 'GD', 'ERR']]
jul_copy_hit = jul_copy_hit.assign(LOB=None, P_HRA_RT=None, P_AB_CN=None, P_HIT_CN=None)

In [23]:
jul_copy_pit = jul_copy[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK',
       'WLS', 'HOLD', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2', 'H3', 'HR', 'SB',
       'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'WP', 'BK', 'ERR', 'R',
       'ER']]

jul_copy_pit = jul_copy_pit.assign(P_WHIP_RT=None, P2_WHIP_RT=None, CB_WHIP_RT=None)

In [24]:
jul_copy_pit['INN2'] = jul_copy_pit['INN2'].astype('str')

In [25]:
for i in range(jul_copy_pit.shape[0]):
    if len(jul_copy_pit.iloc[i, 9].split(' ')) == 1:
        jul_copy_pit.iloc[i, 9] = int(jul_copy_pit.iloc[i, 9])*3
    else:
        jul_copy_pit.iloc[i, 9] = int(jul_copy_pit.iloc[i, 9].split(' ')[0])*3 + (int(jul_copy_pit.iloc[i, 9].split(' ')[1].split('/')[0])/int(jul_copy_pit.iloc[i, 9].split(' ')[1].split('/')[1]))*3

## Aug

In [26]:
options = Options()
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)

In [27]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('08')
time.sleep(1)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [28]:
aug = pd.read_html(str(soup.select('div.tbl-type06')))[0].iloc[:,:3].reset_index(drop=True)

In [29]:
a = pd.DataFrame(columns=['날짜', '시간', '경기'])
for i in range(aug.shape[0]):
    a = pd.concat([a, pd.DataFrame([aug.iloc[i, :]]), pd.DataFrame([aug.iloc[i, :]])])
aug = a.reset_index(drop=True)
aug = aug.assign(AB=0, PA=0, RBI=0, RUN=0, HIT=0, H2=0, H3=0, HR=0, SB=0, CS=0, SH=0, SF=0, BB=0, IB=0,
           HP=0, KK=0, GD=0, ERR=0, CG_CK=0, WLS=0, HOLD=0, WP=0, BK=0, INN2=0, BF=0, ER=0, R=0)
aug = pd.concat([aug.drop([264, 265, 274, 275]), aug.iloc[[264, 265, 274, 275],]]).reset_index(drop=True)

In [30]:
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)
driver.get('https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx')
driver.find_element_by_css_selector('#contents > div.date-select > ul > li.today > img').click()
driver.find_element_by_css_selector('#ui-datepicker-div > div > div > select.ui-datepicker-month').send_keys('8월(AUG)')
driver.find_element_by_css_selector('#ui-datepicker-div > table > tbody > tr:nth-child(1) > td:nth-child(7)').click()
    
n = 0

for j in range(29):
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    x = len(list(soup.find_all('div', {'class':'game-cont'})))
    
    for i in range(1, x+1):
        try:
            driver.find_element_by_css_selector('#contents > div.today-game > div > div.bx-viewport > ul > li:nth-child(%d) > div' % i).click()
            driver.find_element_by_css_selector('#tabDepth2 > li:nth-child(2) > a').click()
            
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            table = pd.read_html(str(soup.select('div.summary.tbl-type02.mt30.mb30')))[0]
            table.index = table[0]
            table = table.drop([0], axis=1)
            table = table.T.reset_index(drop=True)
            
            T_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[0],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[1],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[2]], axis=1).iloc[:, 2:]
            
            B_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[3],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[4],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[5]], axis=1).iloc[:, 2:]
            
            T_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[0]
            
            B_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[1]
         
            aug.iloc[n, 5] = int(T_hit.loc[T_hit['선수명'].isnull(), '타점'])
            aug.iloc[n, 6] = int(T_hit.loc[T_hit['선수명'].isnull(), '득점'])
            aug.iloc[n, 7] = int(T_hit.loc[T_hit['선수명'].isnull(), '안타'])
            aug.iloc[n+1, 5] = int(B_hit.loc[B_hit['선수명'].isnull(), '타점'])
            aug.iloc[n+1, 6] = int(B_hit.loc[B_hit['선수명'].isnull(), '득점'])
            aug.iloc[n+1, 7] = int(B_hit.loc[B_hit['선수명'].isnull(), '안타'])
            
            for i in range(T_hit.shape[0]):
                for j in range(1, 10):
                    if T_hit.iloc[i, j] == '4구':
                        aug.iloc[n, 15] += 1
                    elif T_hit.iloc[i, j] == '고4':
                        aug.iloc[n+1, 16] += 1
                    elif T_hit.iloc[i, j] == '사구':
                        aug.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(T_hit.iloc[i,j])) != None:
                        aug.iloc[n, aug.columns.get_loc('H2')] += 1
                    elif re.search('[가-힣]3', str(T_hit.iloc[i,j])) != None:
                        aug.iloc[n, aug.columns.get_loc('H3')] += 1
                    elif re.search('[가-힣]홈', str(T_hit.iloc[i,j])) != None:
                        aug.iloc[n, aug.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(T_hit.iloc[i,j])) != None:
                        aug.iloc[n, aug.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(T_hit.iloc[i,j])) != None:
                        aug.iloc[n, aug.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(T_hit.iloc[i,j])) != None:
                        aug.iloc[n, aug.columns.get_loc('GD')] += 1
            
            for i in range(B_hit.shape[0]):
                for j in range(1, 10):
                    if B_hit.iloc[i, j] == '4구':
                        aug.iloc[n+1, 15] += 1
                    elif B_hit.iloc[i, j] == '고4':
                        aug.iloc[n+1, 16] += 1
                    elif B_hit.iloc[i, j] == '사구':
                        aug.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(B_hit.iloc[i,j])) != None:
                        aug.iloc[n+1, aug.columns.get_loc('H2')] +=  1
                    elif re.search('[가-힣]3', str(B_hit.iloc[i,j])) != None:
                        aug.iloc[n+1, aug.columns.get_loc('H3')] +=  1
                    elif re.search('[가-힣]홈', str(B_hit.iloc[i,j])) != None:
                        aug.iloc[n+1, aug.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(B_hit.iloc[i,j])) != None:
                        aug.iloc[n+1, aug.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(B_hit.iloc[i,j])) != None:
                        aug.iloc[n+1, aug.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(B_hit.iloc[i,j])) != None:
                        aug.iloc[n+1, aug.columns.get_loc('GD')] += 1
            
            if '도루' in table.columns:
                for i in range(len(table['도루'][0].split(' '))):
                    if table['도루'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        aug.iloc[n, 11] += 1
                    elif table['도루'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        aug.iloc[n+1, 11] += 1
            else: pass
            
            if '도루자' in table.columns:
                for i in range(len(table['도루자'][0].split(' '))):
                    if table['도루자'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        aug.iloc[n, 12] = aug.iloc[n, 12] + 1
                    elif table['도루자'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        aug.iloc[n+1, 12] = aug.iloc[n+1, 12] + 1
            else: pass
            
            if '실책' in table.columns:
                for i in range(len(table['실책'][0].split(' '))):
                    if table['실책'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        aug.iloc[n, 20] += 1
                    elif table['실책'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        aug.iloc[n+1, 20] += 1
            
            # CG_CK
            if T_pit.shape[0] == 2:
                aug.iloc[n, 21] = 1
            else: pass
            if B_pit.shape[0] == 2:
                aug.iloc[n+1, 21] = 1
            else: pass
            
            if '승' in list(T_pit.iloc[:,2]):
                aug.iloc[n, 22] = 'W'
                aug.iloc[n+1, 22] = 'L'
            elif '패' in list(T_pit.iloc[:,2]):
                aug.iloc[n, 22] = 'L'
                aug.iloc[n+1, 22] = 'W'
            else:
                aug.iloc[n, 22] = 'D'
                aug.iloc[n+1, 22] = 'D'
            
            if '폭투' in table.columns:
                for i in range(len(table['폭투'][0].split(' '))):
                    if table['폭투'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        aug.iloc[n, 24] += 1
                    elif table['폭투'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        aug.iloc[n+1, 24] += 1
            
            if '보크' in table.columns:
                for i in range(len(table['보크'][0].split(' '))):
                    if table['보크'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        aug.iloc[n, 25] += 1
                    elif table['보크'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        aug.iloc[n+1, 25] += 1  
            
            if '홀드' in list(T_pit.iloc[:,2]):
                    aug.iloc[n, 23] = list(T_pit.iloc[:,2]).count('홀드')
            if '홀드' in list(B_pit.iloc[:,2]):
                aug.iloc[n+1, 23] = list(B_pit.iloc[:,2]).count('홀드')
            
            # KK
            aug.iloc[n, 18] = T_pit.iloc[-1, 13]
            aug.iloc[n+1, 18] = B_pit.iloc[-1, 13]
            
            # INN2
            aug.iloc[n, 26] = T_pit.iloc[-1, 6]
            aug.iloc[n+1, 26] = B_pit.iloc[-1, 6]
            
            # BF
            aug.iloc[n, 27] = T_pit.iloc[-1, 8]
            aug.iloc[n+1, 27] = B_pit.iloc[-1, 8]
            
            # ER
            aug.iloc[n, 28] = T_pit.iloc[-1, 15]
            aug.iloc[n+1, 28] = B_pit.iloc[-1, 15]
            
            # R
            aug.iloc[n, 29] = T_pit.iloc[-1, 14]
            aug.iloc[n+1, 29] = B_pit.iloc[-1, 14]
            
            # PA
            aug.iloc[n, 4] = B_pit.iloc[-1, 7]
            aug.iloc[n+1, 4] = T_pit.iloc[-1, 7]
            
            # AB
            aug.iloc[n, 3] = B_pit.iloc[-1, 9]
            aug.iloc[n+1, 3] = T_pit.iloc[-1, 9]
            
            try: 
                driver.find_element_by_css_selector('#contents > div.today-game > div > div.bx-controls.bx-has-controls-direction > div > a.bx-next').click()
            except: pass
            
        except: pass
        
        n += 2

    driver.find_element_by_css_selector('#lnkNext > img').click()

In [31]:
aug_copy = aug.copy()

In [32]:
for i in range(aug_copy.shape[0]-10):
    for j in range(1, 11):
        if sum(aug_copy.iloc[i, 3:] == aug_copy.iloc[i+j, 3:]) == 27:
            aug_copy.iloc[i+j, 3:] = 0

In [33]:
aug_copy_a = aug_copy.iloc[[2*n for n in range(int(aug_copy.shape[0]/2))],].reset_index(drop=True)
aug_copy_b = aug_copy.iloc[[2*n+1 for n in range(int(aug_copy.shape[0]/2))],].reset_index(drop=True)

In [34]:
aug_copy_a = aug_copy_a.assign(T=None, B=None)
aug_copy_b = aug_copy_b.assign(T=None, B=None)

In [35]:
for i in range(aug_copy_a.shape[0]):
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('T')] = aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('경기')].split('vs')[0][:2]
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('B')] = aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('경기')].split('vs')[1][-2:]
for i in range(aug_copy_b.shape[0]):
    aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('T')] = aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('경기')].split('vs')[0][:2]
    aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('B')] = aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('경기')].split('vs')[1][-2:]

In [36]:
for i in range(aug_copy_a.shape[0]):
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('날짜')] = '2020.' + aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('날짜')][:5]
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('날짜')] = aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('날짜')].replace('.', '')
for i in range(aug_copy_b.shape[0]):
    aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('날짜')] = '2020.' + aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('날짜')][:5]
    aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('날짜')] = aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('날짜')].replace('.', '')

In [37]:
name = {'NC':'NC', '두산':'OB', '키움':'WO', 'KI':'HT', 'IA':'HT', 'LG':'LG',
       '삼성':'SS', 'KT':'KT', '롯데':'LT', 'SK':'SK', '한화':'HH'}

f = lambda x: name.get(x, x)
aug_copy_a['T'] = aug_copy_a['T'].map(f) ; aug_copy_a['B'] = aug_copy_a['B'].map(f)
aug_copy_b['T'] = aug_copy_b['T'].map(f) ; aug_copy_b['B'] = aug_copy_b['B'].map(f)

In [38]:
aug_copy_a['G_ID'] = aug_copy_a['날짜'] + aug_copy_a['T'] + aug_copy_a['B']
aug_copy_b['G_ID'] = aug_copy_b['날짜'] + aug_copy_b['T'] + aug_copy_b['B']

In [39]:
for i in aug_copy_a[aug_copy_a.duplicated(['G_ID'], keep=False) == True].index:
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('G_ID')] = aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('G_ID')] + '1'
for i in aug_copy_a[aug_copy_a.duplicated(['G_ID'], keep=False) == False].index:
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('G_ID')] = aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('G_ID')] + '0'
for i in aug_copy_b[aug_copy_b.duplicated(['G_ID'], keep=False) == True].index:
    aug_copy_b.iloc[i, aug_copy_b.columns.get_loc('G_ID')] = aug_copy_b.iloc[i, aug_copy_b.columns.get_loc('G_ID')] + '1'
for i in aug_copy_b[aug_copy_b.duplicated(['G_ID'], keep=False) == False].index:
    aug_copy_b.iloc[i, aug_copy_b.columns.get_loc('G_ID')] = aug_copy_b.iloc[i, aug_copy_b.columns.get_loc('G_ID')] + '0'

In [40]:
aug_copy_a = aug_copy_a.drop(['시간', '경기'], axis=1)
aug_copy_b = aug_copy_b.drop(['시간', '경기'], axis=1)

In [41]:
aug_copy_a = aug_copy_a.rename(columns={'T':'T_ID', 'B':'VS_T_ID'})
aug_copy_b = aug_copy_b.rename(columns={'T':'VS_T_ID', 'B':'T_ID'})

In [42]:
aug_copy_a = aug_copy_a.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)
aug_copy_b = aug_copy_b.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)

In [43]:
for i in range(aug_copy_a.shape[0]):
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('GDAY_DS')] = aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('G_ID')][:8]
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('HEADER_NO')] = aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('G_ID')][12]
    aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('TB_SC')] = 'T'
for i in range(aug_copy_b.shape[0]):
    aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('GDAY_DS')] = aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('G_ID')][:8]
    aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('HEADER_NO')] = aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('G_ID')][12]
    aug_copy_b.iloc[i, aug_copy_a.columns.get_loc('TB_SC')] = 'B'

In [44]:
for i in range(aug_copy_a.shape[0]):
    if aug_copy_a.iloc[i, aug_copy_a.columns.get_loc('G_ID')][-1] == '1':
        aug_copy_a.iloc[i+1, aug_copy_a.columns.get_loc('G_ID')] = aug_copy_a.iloc[i+1, aug_copy_a.columns.get_loc('G_ID')].replace('1', '2')
        aug_copy_a.iloc[i+1, aug_copy_a.columns.get_loc('HEADER_NO')] = '2'
for i in range(aug_copy_b.shape[0]):
    if aug_copy_b.iloc[i, aug_copy_b.columns.get_loc('G_ID')][-1] == '1':
        aug_copy_b.iloc[i+1, aug_copy_b.columns.get_loc('G_ID')] = aug_copy_b.iloc[i+1, aug_copy_b.columns.get_loc('G_ID')].replace('1', '2')
        aug_copy_b.iloc[i+1, aug_copy_b.columns.get_loc('HEADER_NO')] = '2'

In [45]:
aug_copy = pd.concat([aug_copy_a, aug_copy_b]).sort_values('G_ID').reset_index(drop=True)

In [46]:
aug_copy_hit = aug_copy[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'PA', 'AB',
       'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
       'IB', 'HP', 'KK', 'GD', 'ERR']]
aug_copy_hit = aug_copy_hit.assign(LOB=None, P_HRA_RT=None, P_AB_CN=None, P_HIT_CN=None)

In [47]:
aug_copy_pit = aug_copy[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK',
       'WLS', 'HOLD', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2', 'H3', 'HR', 'SB',
       'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'WP', 'BK', 'ERR', 'R',
       'ER']]

aug_copy_pit = aug_copy_pit.assign(P_WHIP_RT=None, P2_WHIP_RT=None, CB_WHIP_RT=None)

In [48]:
aug_copy_pit['INN2'] = aug_copy_pit['INN2'].astype('str')

In [49]:
for i in range(aug_copy_pit.shape[0]):
    if len(aug_copy_pit.iloc[i, 9].split(' ')) == 1:
        aug_copy_pit.iloc[i, 9] = int(aug_copy_pit.iloc[i, 9])*3
    else:
        aug_copy_pit.iloc[i, 9] = int(aug_copy_pit.iloc[i, 9].split(' ')[0])*3 + (int(aug_copy_pit.iloc[i, 9].split(' ')[1].split('/')[0])/int(aug_copy_pit.iloc[i, 9].split(' ')[1].split('/')[1]))*3

## Sep

In [50]:
options = Options()
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)

In [51]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('09')
time.sleep(1)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [52]:
sep = pd.read_html(str(soup.select('div.tbl-type06')))[0].iloc[:127,:3].reset_index(drop=True)

In [53]:
a = pd.DataFrame(columns=['날짜', '시간', '경기'])
for i in range(sep.shape[0]):
    a = pd.concat([a, pd.DataFrame([sep.iloc[i, :]]), pd.DataFrame([sep.iloc[i, :]])])
sep = a.reset_index(drop=True)
sep = sep.assign(AB=0, PA=0, RBI=0, RUN=0, HIT=0, H2=0, H3=0, HR=0, SB=0, CS=0, SH=0, SF=0, BB=0, IB=0,
           HP=0, KK=0, GD=0, ERR=0, CG_CK=0, WLS=0, HOLD=0, WP=0, BK=0, INN2=0, BF=0, ER=0, R=0)
sep = sep.iloc[list(range(30))+list(range(38, 44))+list(range(30, 38))+list(range(44, 78))+list(range(82, 90))
         +list(range(78, 82))+list(range(90, 254)),].reset_index(drop=True)

In [54]:
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)
driver.get('https://www.koreabaseball.com/Schedule/GameCenter/Main.aspx')
driver.find_element_by_css_selector('#contents > div.date-select > ul > li.today > img').click()
driver.find_element_by_css_selector('#ui-datepicker-div > div > div > select.ui-datepicker-month').send_keys('9월(sep)')
driver.find_element_by_css_selector('#ui-datepicker-div > table > tbody > tr:nth-child(1) > td:nth-child(3)').click()

n = 0

for j in range(26):
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    x = len(list(soup.find_all('div', {'class':'game-cont'})))
    
    for i in range(1, x+1):
        try:
            driver.find_element_by_css_selector('#contents > div.today-game > div > div.bx-viewport > ul > li:nth-child(%d) > div' % i).click()
            driver.find_element_by_css_selector('#tabDepth2 > li:nth-child(2) > a').click()
            
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            table = pd.read_html(str(soup.select('div.summary.tbl-type02.mt30.mb30')))[0]
            table.index = table[0]
            table = table.drop([0], axis=1)
            table = table.T.reset_index(drop=True)
            
            T_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[0],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[1],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[2]], axis=1).iloc[:, 2:]
            
            B_hit = pd.concat([pd.read_html(str(soup.select('div.hitter-record-area')))[3],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[4],
                               pd.read_html(str(soup.select('div.hitter-record-area')))[5]], axis=1).iloc[:, 2:]
            
            T_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[0]
            
            B_pit = pd.read_html(str(soup.select('div.pitcher-record-area')))[1]
         
            sep.iloc[n, 3] = int(T_hit.loc[T_hit['선수명'].isnull(), '타수'])
            sep.iloc[n, 5] = int(T_hit.loc[T_hit['선수명'].isnull(), '타점'])
            sep.iloc[n, 6] = int(T_hit.loc[T_hit['선수명'].isnull(), '득점'])
            sep.iloc[n, 7] = int(T_hit.loc[T_hit['선수명'].isnull(), '안타'])
            sep.iloc[n+1, 3] = int(B_hit.loc[B_hit['선수명'].isnull(), '타수'])
            sep.iloc[n+1, 5] = int(B_hit.loc[B_hit['선수명'].isnull(), '타점'])
            sep.iloc[n+1, 6] = int(B_hit.loc[B_hit['선수명'].isnull(), '득점'])
            sep.iloc[n+1, 7] = int(B_hit.loc[B_hit['선수명'].isnull(), '안타'])
            
            for i in range(T_hit.shape[0]):
                for j in range(1, 10):
                    if T_hit.iloc[i, j] == '4구':
                        sep.iloc[n, 15] += 1
                    elif T_hit.iloc[i, j] == '고4':
                        sep.iloc[n+1, 16] += 1
                    elif T_hit.iloc[i, j] == '사구':
                        sep.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(T_hit.iloc[i,j])) != None:
                        sep.iloc[n, sep.columns.get_loc('H2')] += 1
                    elif re.search('[가-힣]3', str(T_hit.iloc[i,j])) != None:
                        sep.iloc[n, sep.columns.get_loc('H3')] += 1
                    elif re.search('[가-힣]홈', str(T_hit.iloc[i,j])) != None:
                        sep.iloc[n, sep.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(T_hit.iloc[i,j])) != None:
                        sep.iloc[n, sep.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(T_hit.iloc[i,j])) != None:
                        sep.iloc[n, sep.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(T_hit.iloc[i,j])) != None:
                        sep.iloc[n, sep.columns.get_loc('GD')] += 1
            
            for i in range(B_hit.shape[0]):
                for j in range(1, 10):
                    if B_hit.iloc[i, j] == '4구':
                        sep.iloc[n+1, 15] += 1
                    elif B_hit.iloc[i, j] == '고4':
                        sep.iloc[n+1, 16] += 1
                    elif B_hit.iloc[i, j] == '사구':
                        sep.iloc[n+1, 17] += 1
                for j in range(1, 10):
                    if re.search('[가-힣]2', str(B_hit.iloc[i,j])) != None:
                        sep.iloc[n+1, sep.columns.get_loc('H2')] +=  1
                    elif re.search('[가-힣]3', str(B_hit.iloc[i,j])) != None:
                        sep.iloc[n+1, sep.columns.get_loc('H3')] +=  1
                    elif re.search('[가-힣]홈', str(B_hit.iloc[i,j])) != None:
                        sep.iloc[n+1, sep.columns.get_loc('HR')] += 1
                    elif re.search('[가-힣]희번', str(B_hit.iloc[i,j])) != None:
                        sep.iloc[n+1, sep.columns.get_loc('SH')] += 1
                    elif re.search('[가-힣]희비', str(B_hit.iloc[i,j])) != None:
                        sep.iloc[n+1, sep.columns.get_loc('SF')] += 1
                    elif re.search('[가-힣]병', str(B_hit.iloc[i,j])) != None:
                        sep.iloc[n+1, sep.columns.get_loc('GD')] += 1
            
            if '도루' in table.columns:
                for i in range(len(table['도루'][0].split(' '))):
                    if table['도루'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        sep.iloc[n, 11] += 1
                    elif table['도루'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        sep.iloc[n+1, 11] += 1
            else: pass
            
            if '도루자' in table.columns:
                for i in range(len(table['도루자'][0].split(' '))):
                    if table['도루자'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        sep.iloc[n, 12] = sep.iloc[n, 12] + 1
                    elif table['도루자'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        sep.iloc[n+1, 12] = sep.iloc[n+1, 12] + 1
            else: pass
            
            if '실책' in table.columns:
                for i in range(len(table['실책'][0].split(' '))):
                    if table['실책'][0].split(' ')[i][:-4] in list(T_hit['선수명']):
                        sep.iloc[n, 20] += 1
                    elif table['실책'][0].split(' ')[i][:-4] in list(B_hit['선수명']):
                        sep.iloc[n+1, 20] += 1
            
            # CG_CK
            if T_pit.shape[0] == 2:
                sep.iloc[n, 21] = 1
            else: pass
            if B_pit.shape[0] == 2:
                sep.iloc[n+1, 21] = 1
            else: pass
            
            if '승' in list(T_pit.iloc[:,2]):
                sep.iloc[n, 22] = 'W'
                sep.iloc[n+1, 22] = 'L'
            elif '패' in list(T_pit.iloc[:,2]):
                sep.iloc[n, 22] = 'L'
                sep.iloc[n+1, 22] = 'W'
            else:
                sep.iloc[n, 22] = 'D'
                sep.iloc[n+1, 22] = 'D'
            
            if '폭투' in table.columns:
                for i in range(len(table['폭투'][0].split(' '))):
                    if table['폭투'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        sep.iloc[n, 24] += 1
                    elif table['폭투'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        sep.iloc[n+1, 24] += 1
            
            if '보크' in table.columns:
                for i in range(len(table['보크'][0].split(' '))):
                    if table['보크'][0].split(' ')[i][:-4] in list(T_pit['선수명']):
                        sep.iloc[n, 25] += 1
                    elif table['보크'][0].split(' ')[i][:-4] in list(B_pit['선수명']):
                        sep.iloc[n+1, 25] += 1  
            
            if '홀드' in list(T_pit.iloc[:,2]):
                    sep.iloc[n, 23] = list(T_pit.iloc[:,2]).count('홀드')
            if '홀드' in list(B_pit.iloc[:,2]):
                sep.iloc[n+1, 23] = list(B_pit.iloc[:,2]).count('홀드')
            
            # KK
            sep.iloc[n, 18] = T_pit.iloc[-1, 13]
            sep.iloc[n+1, 18] = B_pit.iloc[-1, 13]
            
            # INN2
            sep.iloc[n, 26] = T_pit.iloc[-1, 6]
            sep.iloc[n+1, 26] = B_pit.iloc[-1, 6]
            
            # BF
            sep.iloc[n, 27] = T_pit.iloc[-1, 8]
            sep.iloc[n+1, 27] = B_pit.iloc[-1, 8]
            
            # ER
            sep.iloc[n, 28] = T_pit.iloc[-1, 15]
            sep.iloc[n+1, 28] = B_pit.iloc[-1, 15]
            
            # R
            sep.iloc[n, 29] = T_pit.iloc[-1, 14]
            sep.iloc[n+1, 29] = B_pit.iloc[-1, 14]
            
            # PA
            sep.iloc[n, 4] = B_pit.iloc[-1, 7]
            sep.iloc[n+1, 4] = T_pit.iloc[-1, 7]
            
            # AB
            sep.iloc[n, 3] = B_pit.iloc[-1, 9]
            sep.iloc[n+1, 3] = T_pit.iloc[-1, 9]
            
            try: 
                driver.find_element_by_css_selector('#contents > div.today-game > div > div.bx-controls.bx-has-controls-direction > div > a.bx-next').click()
            except: pass
            
        except: pass
        
        n += 2

    driver.find_element_by_css_selector('#lnkNext > img').click()

In [55]:
sep_copy = sep.copy()

In [56]:
for i in range(sep_copy.shape[0]-10):
    for j in range(1, 11):
        if sum(sep_copy.iloc[i, 3:] == sep_copy.iloc[i+j, 3:]) == 27:
            sep_copy.iloc[i+j, 3:] = 0

In [57]:
sep_copy_a = sep_copy.iloc[[2*n for n in range(int(sep_copy.shape[0]/2))],].reset_index(drop=True)
sep_copy_b = sep_copy.iloc[[2*n+1 for n in range(int(sep_copy.shape[0]/2))],].reset_index(drop=True)

In [58]:
sep_copy_a = sep_copy_a.assign(T=None, B=None)
sep_copy_b = sep_copy_b.assign(T=None, B=None)

In [59]:
for i in range(sep_copy_a.shape[0]):
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('T')] = sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('경기')].split('vs')[0][:2]
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('B')] = sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('경기')].split('vs')[1][-2:]
for i in range(sep_copy_b.shape[0]):
    sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('T')] = sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('경기')].split('vs')[0][:2]
    sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('B')] = sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('경기')].split('vs')[1][-2:]

In [60]:
for i in range(sep_copy_a.shape[0]):
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('날짜')] = '2020.' + sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('날짜')][:5]
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('날짜')] = sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('날짜')].replace('.', '')
for i in range(sep_copy_b.shape[0]):
    sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('날짜')] = '2020.' + sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('날짜')][:5]
    sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('날짜')] = sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('날짜')].replace('.', '')

In [61]:
name = {'NC':'NC', '두산':'OB', '키움':'WO', 'KI':'HT', 'IA':'HT', 'LG':'LG',
       '삼성':'SS', 'KT':'KT', '롯데':'LT', 'SK':'SK', '한화':'HH'}

f = lambda x: name.get(x, x)
sep_copy_a['T'] = sep_copy_a['T'].map(f) ; sep_copy_a['B'] = sep_copy_a['B'].map(f)
sep_copy_b['T'] = sep_copy_b['T'].map(f) ; sep_copy_b['B'] = sep_copy_b['B'].map(f)

In [62]:
sep_copy_a['G_ID'] = sep_copy_a['날짜'] + sep_copy_a['T'] + sep_copy_a['B']
sep_copy_b['G_ID'] = sep_copy_b['날짜'] + sep_copy_b['T'] + sep_copy_b['B']

In [63]:
for i in sep_copy_a[sep_copy_a.duplicated(['G_ID'], keep=False) == True].index:
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')] = sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')] + '1'
for i in sep_copy_a[sep_copy_a.duplicated(['G_ID'], keep=False) == False].index:
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')] = sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')] + '0'
for i in sep_copy_b[sep_copy_b.duplicated(['G_ID'], keep=False) == True].index:
    sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')] = sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')] + '1'
for i in sep_copy_b[sep_copy_b.duplicated(['G_ID'], keep=False) == False].index:
    sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')] = sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')] + '0'

In [64]:
sep_copy_a = sep_copy_a.drop(['시간', '경기'], axis=1)
sep_copy_b = sep_copy_b.drop(['시간', '경기'], axis=1)

In [65]:
sep_copy_a = sep_copy_a.rename(columns={'T':'T_ID', 'B':'VS_T_ID'})
sep_copy_b = sep_copy_b.rename(columns={'T':'VS_T_ID', 'B':'T_ID'})

In [66]:
sep_copy_a = sep_copy_a.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)
sep_copy_b = sep_copy_b.assign(GDAY_DS=None, HEADER_NO=None, TB_SC=None)

In [67]:
for i in range(sep_copy_a.shape[0]):
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('GDAY_DS')] = sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')][:8]
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('HEADER_NO')] = sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')][12]
    sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('TB_SC')] = 'T'
for i in range(sep_copy_b.shape[0]):
    sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('GDAY_DS')] = sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('G_ID')][:8]
    sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('HEADER_NO')] = sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('G_ID')][12]
    sep_copy_b.iloc[i, sep_copy_a.columns.get_loc('TB_SC')] = 'B'

In [68]:
for i in range(sep_copy_a.shape[0]-1):
    if (sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')][-1] == '1') & (sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')] == sep_copy_a.iloc[i+1, sep_copy_a.columns.get_loc('G_ID')]):
        sep_copy_a.iloc[i+1, sep_copy_a.columns.get_loc('G_ID')] = sep_copy_a.iloc[i+1, sep_copy_a.columns.get_loc('G_ID')].replace('1', '2')
        sep_copy_a.iloc[i+1, sep_copy_a.columns.get_loc('HEADER_NO')] = '2'
    elif (sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')][-1] == '1') & (sep_copy_a.iloc[i, sep_copy_a.columns.get_loc('G_ID')] != sep_copy_a.iloc[i+1, sep_copy_a.columns.get_loc('G_ID')]):
        sep_copy_a.iloc[i+2, sep_copy_a.columns.get_loc('G_ID')] = sep_copy_a.iloc[i+2, sep_copy_a.columns.get_loc('G_ID')].replace('1', '2')
        sep_copy_a.iloc[i+2, sep_copy_a.columns.get_loc('HEADER_NO')] = '2'

for i in range(sep_copy_b.shape[0]-1):
    if (sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')][-1] == '1') & (sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')] == sep_copy_b.iloc[i+1, sep_copy_b.columns.get_loc('G_ID')]):
        sep_copy_b.iloc[i+1, sep_copy_b.columns.get_loc('G_ID')] = sep_copy_b.iloc[i+1, sep_copy_b.columns.get_loc('G_ID')].replace('1', '2')
        sep_copy_b.iloc[i+1, sep_copy_b.columns.get_loc('HEADER_NO')] = '2'
    elif (sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')][-1] == '1') & (sep_copy_b.iloc[i, sep_copy_b.columns.get_loc('G_ID')] != sep_copy_b.iloc[i+1, sep_copy_b.columns.get_loc('G_ID')]):
        sep_copy_b.iloc[i+2, sep_copy_b.columns.get_loc('G_ID')] = sep_copy_b.iloc[i+2, sep_copy_b.columns.get_loc('G_ID')].replace('1', '2')
        sep_copy_b.iloc[i+2, sep_copy_b.columns.get_loc('HEADER_NO')] = '2'

In [69]:
sep_copy = pd.concat([sep_copy_a, sep_copy_b]).sort_values('G_ID').reset_index(drop=True)

In [70]:
sep_copy_hit = sep_copy[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'PA', 'AB',
       'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
       'IB', 'HP', 'KK', 'GD', 'ERR']]
sep_copy_hit = sep_copy_hit.assign(LOB=None, P_HRA_RT=None, P_AB_CN=None, P_HIT_CN=None)

In [71]:
sep_copy_pit = sep_copy[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK',
       'WLS', 'HOLD', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2', 'H3', 'HR', 'SB',
       'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'WP', 'BK', 'ERR', 'R',
       'ER']]

sep_copy_pit = sep_copy_pit.assign(P_WHIP_RT=None, P2_WHIP_RT=None, CB_WHIP_RT=None)

In [72]:
sep_copy_pit['INN2'] = sep_copy_pit['INN2'].astype('str')

In [73]:
for i in range(sep_copy_pit.shape[0]):
    if len(sep_copy_pit.iloc[i, 9].split(' ')) == 1:
        sep_copy_pit.iloc[i, 9] = int(sep_copy_pit.iloc[i, 9])*3
    else:
        sep_copy_pit.iloc[i, 9] = int(sep_copy_pit.iloc[i, 9].split(' ')[0])*3 + (int(sep_copy_pit.iloc[i, 9].split(' ')[1].split('/')[0])/int(sep_copy_pit.iloc[i, 9].split(' ')[1].split('/')[1]))*3

## Total

In [74]:
chit20 = pd.concat([jul_copy_hit, aug_copy_hit, sep_copy_hit]).sort_values('G_ID').reset_index(drop=True)
cpit20 = pd.concat([jul_copy_pit, aug_copy_pit, sep_copy_pit]).sort_values('G_ID').reset_index(drop=True) ; cpit20.INN2 = cpit20.INN2.astype('int')

In [75]:
chit20.to_csv('chit20.csv', index=False)
cpit20.to_csv('cpit20.csv', index=False)